### Steps for downloading GBIF data for another species for your portfolio post

#### Step 1: choose a terrestrial species that migrates (and find its scientific/Latin name)

#### Step 2: make an account on [gbif.org](<https://www.gbif.org/)>)

#### Step 3: Install the pygbif package in your python environment

If you're on a Mac, just run this code chunk:

In [ ]:
# PC specific install required
# pip install pygbif

#%% bash
#pip install pygbif

UsageError: Cell magic `%%` not found.


If you're on a PC, open Git Bash in the Terminal at the bottom of the Jupyter notebook, and enter this: 

`pip install pygbif`

#### Step 4: load the packages you'll need

In [21]:
import pathlib
import os
import pygbif.occurrences as occ
import pygbif.species as species
from getpass import getpass
from glob import glob
import time
import zipfile

#### Step 5: make a folder for your GBIF data
You'll need to modify this code to match your folder structure:

In [2]:
### create data directory
pluv_dir = os.path.join(

    ### home directory
    pathlib.Path.home(),

    ### data directory within migration git repository
    'Documents',
    'Earth Data Cert',
    'Earth-Analytics-AY25',
    'GitRepos',
    '03-migration-natshoff',
    'data')

### make the directory
os.makedirs(pluv_dir, exist_ok = True)

### define directory for the American Golden Plover gbif data
gbif_dir = os.path.join(pluv_dir, 'pluvialisDominica_gbif')

### make the directory
os.makedirs(gbif_dir, exist_ok = True)

#### Step 6: link GBIF to your Jupyter notebook
- Change to `reset = True` the first time you run it. When you run it, you'll be prompted to enter your GBIF username, password, and associated email address
- Then change it back to `reset = False` so you can re-run the chunk without having to reconnect to GBIF
- You don’t need to modify this code chunk in any other way


In [ ]:
####--------------------------####
#### DO NOT MODIFY THIS CODE! ####
####--------------------------####
# This code ASKS for your credentials 
# and saves it for the rest of the session.
# NEVER put your credentials into your code!!!!

# GBIF needs a username, password, and email 
# All 3 need to match the account
reset = False

# Request and store username
if (not ('GBIF_USER'  in os.environ)) or reset:
    os.environ['GBIF_USER'] = input('GBIF username:')

# Securely request and store password
if (not ('GBIF_PWD'  in os.environ)) or reset:
    os.environ['GBIF_PWD'] = getpass('GBIF password:')
    
# Request and store account email address
if (not ('GBIF_EMAIL'  in os.environ)) or reset:
    os.environ['GBIF_EMAIL'] = input('GBIF email:')

#### Step 7: Get the species key for your species from GBIF using the `name_backbone` command
You'll need to modify this code with the scientific name of your species

In [15]:
### grab the species info
backbone = species.name_backbone(name = 'Pluvialis dominica')

### check it out
print(backbone)

### pull out the species key
species_key = backbone['usageKey']

### check it out
print("Usage key:", species_key)

{'usageKey': 2480328, 'scientificName': 'Pluvialis dominica (P.L.S.Müller, 1776)', 'canonicalName': 'Pluvialis dominica', 'rank': 'SPECIES', 'status': 'ACCEPTED', 'confidence': 99, 'matchType': 'EXACT', 'kingdom': 'Animalia', 'phylum': 'Chordata', 'order': 'Charadriiformes', 'family': 'Charadriidae', 'genus': 'Pluvialis', 'species': 'Pluvialis dominica', 'kingdomKey': 1, 'phylumKey': 44, 'classKey': 212, 'orderKey': 7192402, 'familyKey': 2480210, 'genusKey': 2480326, 'speciesKey': 2480328, 'class': 'Aves'}
Usage key: 2480328


#### Step 8: Run this code to download the GBIF data once
It might take several minutes for the data to download

Modify this code to update the year if you want data for a different year, and to use the correct data_dir for your file structure (my code says "gbif_dir")

In [22]:
# Only download once
### set file name for download
gbif_pattern = os.path.join(gbif_dir, '*.csv')

### double check that there isn't already a file that matches this pattern.
### if it already exists, skip the whole conditional
### and go straight to the line: gbif_path = glob(gbif_pattern)[0]
if not glob(gbif_pattern):

    ### only submit a download request to GBIF once
    ### if GBIF_DOWNLOAD_KEY is not defined in our environment, make the download request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:

        ### submit a query to GBIF
        gbif_query = occ.download([

            ### add your species key here
            f"speciesKey = {species_key}",

            ### filter out results that are missing coordinates
            "hasCoordinate = True",

            ### choose a year to include
            "year = 2023",
        ])
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    download_key = os.environ['GBIF_DOWNLOAD_KEY']

    ### use the occurrence command module in pygbif to get the metadata
    wait = occ.download_meta(download_key)['status']

    ### check if the status of the download = "SUCCEEDED"
    ### wait and loop through until it finishes
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']

        ### don't want to re-query the API in the loop too frequently
        time.sleep(5)

    # Download GBIF data when it's ready
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=gbif_dir)

    # Unzip GBIF data using the zipfile package
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path (take the first result)
gbif_path = glob(gbif_pattern)[0]

INFO:Download file size: 1817263 bytes
INFO:On disk at C:\Users\naho5798\Documents\Earth Data Cert\Earth-Analytics-AY25\GitRepos\03-migration-natshoff\data\pluvialisDominica_gbif/0006747-251025141854904.zip


The GBIF occurrence data should now be in a folder on your computer, saved as a .csv file. You can open it using the code structure we've used before to open .csv files using pandas.